## Hello World using Flask

In [1]:
import os
script_file = os.path.join(os.path.pardir, "src", "models", "hello_world_api.py")

In [2]:
%%writefile $script_file
from flask import Flask, request

app = Flask(__name__)

@app.route("/api", methods=["POST"])
def say_hello():
    data = request.get_json(force=True)
    name = data["name"]
    return "Hello, {0}".format(name)

if __name__ == "__main__":
    app.run(port=9001, debug=True)

Writing ..\src\models\hello_world_api.py


In [3]:
import json
import requests

In [4]:
url = "http://localhost:9001/api"
data = json.dumps({"name": "Rahul"})
r = requests.post(url, data)

In [5]:
print(r.text)

Hello, Rahul


## ML API

In [6]:
ml_api_file = os.path.join(os.path.pardir, "src", "models", "ml_api.py")

In [14]:
%%writefile $ml_api_file
from flask import Flask, request
import pandas as pd
import numpy as np
import pickle
import json
import os

app = Flask(__name__)

# Load model and scaler
model_path = os.path.join(os.path.pardir, os.path.pardir, "models")
model_file = os.path.join(model_path, "lr_model.pkl")
scaler_file = os.path.join(model_path, "lr_scaler.pkl")

scaler = pickle.load(open(scaler_file, "rb"))
model = pickle.load(open(model_file, "rb"))

columns = ["Age", "Fare", "FamilySize",
       "IsMother", "IsMale", "Deck_A", "Deck_B", "Deck_C", "Deck_D", 
       "Deck_E", "Deck_F", "Deck_G", "Deck_Z", "Pclass_1", "Pclass_2", 
       "Pclass_3", "Title_Lady", "Title_Master", "Title_Miss", "Title_Mr", 
       "Title_Mrs", "Title_Officer", "Title_Sir", "Fare_Bin_Very_Low", 
       "Fare_Bin_Low", "Fare_Bin_High", "Fare_Bin_Very_High", "Embarked_C", 
       "Embarked_Q", "Embarked_S", "AgeState_Adult", "AgeState_Child"]

@app.route("/api", methods=["POST"])
def make_prediction():
    # Read JSON from the request, convert to JSON string
    data = json.dumps(request.get_json(force=True))
    
    # Create Pandas DataFrame
    df = pd.read_json(data)
    
    # Extract PassengerId
    pids = df["PassengerId"].ravel()
    
    X = df[columns].as_matrix().astype("float")
    X_scaled = scaler.transform(X)
    
    predictions = model.predict(X_scaled)
    df_response = pd.DataFrame({"PassengerId": pids, "Predicted": predictions})
    
    return df_response.to_json()

if __name__ == "__main__":
    app.run(port=9001, debug=True)

Overwriting ..\src\models\ml_api.py


### Invoking ML API

In [8]:
import os
import pandas as pd

In [9]:
processed_data_path = os.path.join(os.path.pardir, "data", "processed")
train_file_path = os.path.join(processed_data_path, "train.csv")
train_df = pd.read_csv(train_file_path)

In [10]:
survived_passengers = train_df[train_df.Survived == 1][:5]

In [11]:
survived_passengers

,PassengerId,Age,Fare,FamilySize,IsMother,IsMale,Deck_A,Deck_B,Deck_C,Deck_D,...,Fare_Bin_Very_Low,Fare_Bin_Low,Fare_Bin_High,Fare_Bin_Very_High,Embarked_C,Embarked_Q,Embarked_S,AgeState_Adult,AgeState_Child,Survived
1,2,38,71.2833,2,0,0,0,0,1,0,...,0,0,0,1,1,0,0,1,0,1
2,3,26,7.9250,1,0,0,0,0,0,0,...,0,1,0,0,0,0,1,1,0,1
3,4,35,53.1000,2,0,0,0,0,1,0,...,0,0,0,1,0,0,1,1,0,1
8,9,27,11.1333,3,1,0,0,0,0,0,...,0,1,0,0,0,0,1,1,0,1
9,10,14,30.0708,2,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,1,1


In [15]:
import requests

def make_api_req(data):
    url = "http://localhost:9001/api"
    r = requests.post(url, data)
    return r.json()

In [16]:
make_api_req(survived_passengers.to_json())

{'PassengerId': {'0': 2, '1': 3, '2': 4, '3': 9, '4': 10},
 'Predicted': {'0': 1, '1': 1, '2': 1, '3': 1, '4': 1}}

In [17]:
result = make_api_req(train_df.to_json())
df_result = pd.read_json(json.dumps(result))
df_result.head()

,PassengerId,Predicted
0,1,0
1,2,1
10,108,0
100,190,0
101,20,1
